# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [4]:
import pandas as pd

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

df.columns = df.columns.str.lower().str.replace(' ', '_')

df['total_claim_amount'] = pd.to_numeric(df['total_claim_amount'], errors='coerce')
df['monthly_premium_auto'] = pd.to_numeric(df['monthly_premium_auto'], errors='coerce')
df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')

df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%Y-%m-%d', errors='coerce')

df = df.dropna(subset=['total_claim_amount', 'response', 'monthly_premium_auto', 'customer_lifetime_value'])

In [5]:
#1Create a new DataFrame that only includes customers wh

low_claim_yes_response = df[(df['total_claim_amount'] < 1000) & (df['response'] == 'Yes')]

print(low_claim_yes_response.head())

    unnamed:_0 customer       state  customer_lifetime_value response  \
3            3  XL78013      Oregon             22332.439460      Yes   
8            8  FM55990  California              5989.773931      Yes   
15          15  CW49887  California              4626.801093      Yes   
19          19  NJ54277  California              3746.751625      Yes   
27          27  MQ68407      Oregon              4376.363592      Yes   

    coverage education effective_to_date employmentstatus gender  ...  \
3   Extended   College               NaT         Employed      M  ...   
8    Premium   College               NaT         Employed      M  ...   
15     Basic    Master               NaT         Employed      F  ...   
19  Extended   College               NaT         Employed      F  ...   
27   Premium  Bachelor               NaT         Employed      F  ...   

    number_of_open_complaints number_of_policies     policy_type  \
3                         0.0                  2  Corp

In [6]:
#2Using the original Dataframe, analyze

yes_response = df[df['response'] == 'Yes']

grouped = yes_response.groupby(['policy_type', 'gender'])[['monthly_premium_auto', 'customer_lifetime_value']].mean().round(2)

print(grouped)

                       monthly_premium_auto  customer_lifetime_value
policy_type    gender                                               
Corporate Auto F                      94.30                  7712.63
               M                      92.19                  7944.47
Personal Auto  F                      99.00                  8339.79
               M                      91.09                  7448.38
Special Auto   F                      92.31                  7691.58
               M                      86.34                  8247.09


In [7]:
claim_by_group = yes_response.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().round(2)

print(claim_by_group)

policy_type     gender
Corporate Auto  F         433.74
                M         408.58
Personal Auto   F         452.97
                M         457.01
Special Auto    F         453.28
                M         429.53
Name: total_claim_amount, dtype: float64


In [8]:
#3Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

customers_by_state = df['state'].value_counts()
states_over_500 = customers_by_state[customers_by_state > 500]

print(states_over_500)

state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


In [9]:
#4Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

clv_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).round(2)

print(clv_stats)

                                  max      min   median
education            gender                            
Bachelor             F       73225.96  1904.00  5663.50
                     M       67907.27  1898.01  5550.70
College              F       61850.19  1898.68  5616.54
                     M       61134.68  1918.12  5989.77
Doctor               F       44856.11  2395.57  5332.46
                     M       32677.34  2267.60  5610.13
High School or Below F       55277.45  2144.92  6054.18
                     M       83325.38  1940.98  6277.01
Master               F       51016.07  2417.78  5714.17
                     M       50568.26  2272.31  5580.96


# Education Level & Gender Analysis of Customer Lifetime Value
1. Customers with “High School or Below” Education (Male) Have the Highest Maximum CLV
Max CLV: $83,325.38 — the highest across all groups.
Median CLV: $6,277.01 — also the highest median, suggesting this group not only includes some very high-value customers, but they are consistently valuable.
Conclusion: This segment may be underestimated and could be targeted more heavily in future marketing efforts.

2. Bachelor's Degree Holders Have Strong Median CLVs
Female Median: $5,663.50
Male Median: $5,550.70
These are moderately high medians, showing solid lifetime value.
Conclusion: The bachelor’s segment is reliable and worth ongoing engagement, though not the top tier.

3. Doctorate Holders Show Low Maximum and Median CLV
Female Max: $44,856.11 — lower than most groups.
Male Max: $32,677.34 — one of the lowest.
Medians are also in the mid-5000s range.
Conclusion: Despite high education, this group may not yield the highest long-term revenue. Possibly fewer purchases or smaller policy sizes.

4. College-Level Education Shows Gender Parity
Female Median: $5,616.54
Male Median: $5,989.77
CLV is very similar for both genders, suggesting consistent behavior regardless of gender within this educational level.
Conclusion: Marketing strategies here can remain broad and non-segmented by gender.

5. Master's Degree Segment Is Moderate
Medians are around $5,700 for both genders.
Max CLV is not particularly high compared to others.
Conclusion: This is a middle-performing group — stable but not top priority.

# Key Takeaways
- Top-performing segment: Males with High School or Below education.
- Most consistent and reliable segment: Those with Bachelor’s degrees.
- Surprising underperformers: Doctorate holders, both male and female.

In [31]:
#5The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.


df['month_name'] = df['month'].apply(lambda x: pd.to_datetime(str(x), format='%m').strftime('%B'))

grouped = df.groupby(['state', 'month_name']).size().reset_index(name='policies_sold')

top_states = grouped.groupby('state')['policies_sold'].sum().nlargest(3).index.tolist()

df_top_states = grouped[grouped['state'].isin(top_states)]

In [32]:
#6Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

pivot_table = df_top_states.pivot(index='state', columns='month_name', values='policies_sold').fillna(0)

pivot_table = pivot_table.reindex(columns=month_order)

print(pivot_table)

month_name  January  February  March  April  May  June  July  August  \
state                                                                  
Arizona        1008       929    NaN    NaN  NaN   NaN   NaN     NaN   
California     2231      1952    NaN    NaN  NaN   NaN   NaN     NaN   
Oregon         1565      1344    NaN    NaN  NaN   NaN   NaN     NaN   

month_name  September  October  November  December  
state                                               
Arizona           NaN      NaN       NaN       NaN  
California        NaN      NaN       NaN       NaN  
Oregon            NaN      NaN       NaN       NaN  


In [34]:
pivot_table = pivot_table.fillna(0).astype(int)
pivot_table['Total'] = pivot_table.sum(axis=1)
pivot_table = pivot_table.sort_values(by='Total', ascending=False)
print(pivot_table)

month_name  January  February  March  April  May  June  July  August  \
state                                                                  
California     2231      1952      0      0    0     0     0       0   
Oregon         1565      1344      0      0    0     0     0       0   
Arizona        1008       929      0      0    0     0     0       0   

month_name  September  October  November  December  Total  
state                                                      
California          0        0         0         0   8366  
Oregon              0        0         0         0   5818  
Arizona             0        0         0         0   3874  


In [23]:
#7The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

if 'sales_channel' in df.columns:
    response_by_channel = df.groupby('sales_channel')['response'].apply(lambda x: (x == 'Yes').mean()).reset_index()
    response_by_channel['response_rate'] = (response_by_channel['response'] * 100).round(2)
    response_by_channel = response_by_channel.drop(columns='response')

    print(response_by_channel)
else:
    print("No 'sales_channel' column found in dataset.")

  sales_channel  response_rate
0         Agent          19.07
1        Branch          11.38
2   Call Center          10.98
3           Web          11.71
